## Perform necessary imports and define global variables
- Note that simulation csv files will be overwritten if using same simulation variables as a previous attempt...
  - Vars in filename == (number_of_simulations, noise_ratio, number_of_orders)

In [1]:
# Import necessary packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import poisson
from IPython.display import clear_output
import scipy as sc
import os, re, random, math, time

# Define important global variables
batches = 5
number_of_simulations = 20  #more than 20 simulations at a time tends to produce memory issues, refer to batches to add more sims
noise_ratios = [0.05, 0.5, 0.75, 0.8, 0.9, 0.95]
sample_of_orders = 100
number_of_orders = 10000  #total number of order quantities to generate prior to sampling
ratio_poisson_per = 0.95
ratio_poisson_com = 1 - ratio_poisson_per
lam_poisson_per = 1   #lambda for personal poisson distribution (value that distribution is centered around)
lam_poisson_com = 50  #lambda for commercial poisson distribution (value that distribution is centered around)
num_poisson_per = int(number_of_orders * ratio_poisson_per)  #number of small order quantities to generate prior to sampling
num_poisson_com = int(number_of_orders * ratio_poisson_com)  #number of large order quantities to generate prior to sampling

# Read in our file containing product info - Finished Goods and corresponding Parts
df_parts = pd.read_csv('finished_good_parts.csv')

# Define path to OUTPUT folder where our serviceability csv files will be located
PATH_TO_OUTPUT = os.path.join(os.getcwd(), 'Sericeability_Output')
# Make directory if specified path doesn't exist
if not os.path.exists(PATH_TO_OUTPUT):
    os.mkdir(PATH_TO_OUTPUT)

In [2]:
def run_simulations(simulations, noise_list, batches):
    for batch in range(batches):
        # Run simulations for each of our passed in noise_ratios
        for noise in noise_list:
            # Initialize fg_serviceability and pt_serviceability DataFrames
            fg_serviceability = pd.DataFrame()
            pt_serviceability = pd.DataFrame()
            full_serviceability = pd.DataFrame()

            print("=== Beginning Simulations - NoiseRatio {} - Batch {} ===".format(noise, (batch+1)))
            for sim_number in range(simulations):
                start_time = time.time()

                # Generate list of integers 1-sample_of_orders to use as identifying ORDER NUMBER's
                order_number = np.arange(1,(sample_of_orders+1))
                # Generate random list of integers (1-5) as the FINISHED GOOD purchased in each order
                FG = [random.randrange(1, 6, 1) for _ in range(sample_of_orders)]

                '''
                Generate order QUANTITIES (number of Finished Good ordered).
                Due to strange poisson distribution we are trying to emulate we build 2 distributions:
                s - list of smaller order quantities to represent personal orders, w/ poisson dist centered around 3
                s_larger - list of larger order quantities to represent company orders, w/ poisson dist centered around 50
                '''
                s = 1 + np.random.poisson(lam_poisson_per, num_poisson_per) 
                s_larger = np.random.poisson(lam_poisson_com, num_poisson_com)

                # Combine our 2 sets and remove all quantities equal to 0
                s_combined = [*s,*s_larger]
                s_combined[:]=(value for value in s_combined if value != 0)

                # Take a random sample of size (sample_of_orders) from our poisson distribution of order QUANTITIES
                order_quantities = np.random.choice(s_combined, sample_of_orders, replace=False)
                # Clear memory
                del s_combined

                # Turn our 3 lists created above into a DataFrame
                df_orders = pd.DataFrame({'Order Number':order_number, 'Finished Good':FG, 'Order Qty':order_quantities})

                # Combine all of our ORDER and PART information into a full DataFrame of Simulated Order Information
                df_full = df_orders.merge(df_parts, on="Finished Good", how="outer")

                # Create new column 'qty_on_order' to show total number of parts required (some FG require 2 of the same part to build)
                df_full['qty_on_order'] = df_full['Order Qty']*df_full['Quantity']
                # Create null column 'fulfilled_parts' that will be updated later
                df_full['fulfilled_parts'] = np.nan

                # Make new DataFrame to represent INVENTORY on hand
                df_inventory = df_full.groupby('Item_Number')[['qty_on_order']].sum()
                df_inventory.reset_index(level=0, inplace=True)
                # SUM up the qty_on_order grouped by each Item_Number and multiply by noise for 'Inventory' levels
                df_inventory['Inventory'] = df_inventory['qty_on_order'].apply(lambda x: math.ceil (x*noise))

                '''
                Loop through our orders sequentially and attempt to fulfill orders part-by-part.
                Compare the qty_on_order of each part to our inventory levels of that part (or Item_Number).
                If there are enough parts in inventory set 'fulfilled_parts' to True, and subtract required
                qty_on_order from our current inventory.
                Otherwise set our 'fulfilled_parts' to False, and do NOT attempt to deplete inventory levels.
                '''
                for i in range(1,(sample_of_orders+1)):
                    # Find all parts and required quantities for Order Number i (1 through 100) 
                    current_order = df_full[df_full['Order Number'] == i]
                    # Iterate through each part on the current_order
                    for j in range(len(current_order)):
                        part = current_order.iloc[j]['Item_Number']
                        qty = current_order.iloc[j]['qty_on_order']
                        on_hand = df_inventory['Inventory'][df_inventory['Item_Number']==part].values[0]
                        # Check to see if we have enough parts on hand
                        if ((on_hand - qty) > 0):
                            # if so set new column ('Fulfilled') to True
                            df_full.loc[(df_full['Order Number']==i) & (df_full['Item_Number']==part),'fulfilled_parts'] = True
                            # subtract qty (quantity of current part for the current order) from inventory on hand
                            df_inventory.loc[df_inventory['Item_Number']==part, 'Inventory'] = df_inventory['Inventory'][df_inventory['Item_Number']==part] - qty
                            #print(df_parts['Inventory'][df_parts['Item_Number']==part])

                        # If we don't have enough parts set 'Fulfilled' value to False
                        else:
                            df_full.loc[(df_full['Order Number']==i) & (df_full['Item_Number']==part),'fulfilled_parts'] = False
                # Clear memory
                del df_inventory

                # Add new columns for each trial onto our existing fg_ and pt_serviceability
                fg_col_name = "FGS_"+str(sim_number+1).zfill(3)
                pt_col_name = "PTS_" + str(sim_number+1).zfill(3)

                # Group by Order Number and Finished Good and get average of fulfilled_parts column
                fg_fulfillment = df_full.groupby(['Order Number', 'Finished Good'])[['fulfilled_parts']].mean()
                fg_fulfillment.reset_index(level=1, inplace=True)
                fg_fulfillment.reset_index(level=0, inplace=True)

                # Set all 'fulfilled_parts' values less than 1 equal to 0 (i.e. order cannot be fully fulfilled/machine not built)
                fg_fulfillment.loc[(fg_fulfillment['fulfilled_parts'] < 1), 'fulfilled_parts'] = 0

                # For first simulation create our fg_ and pt_serviceability DataFrames
                if sim_number == 0:
                    # FINISHED GOODS level
                    fg_serviceability = fg_fulfillment.groupby('Finished Good')[['fulfilled_parts']].mean()
                    fg_serviceability.reset_index(level=0, inplace=True)
                    fg_serviceability = fg_serviceability.rename(columns={"fulfilled_parts":fg_col_name})
                    del fg_fulfillment
                    # PARTS level
                    pt_serviceability = df_full.groupby(['Item_Number'])[['fulfilled_parts']].mean()
                    pt_serviceability.reset_index(level=0, inplace=True)
                    pt_serviceability = pt_serviceability.rename(columns={"fulfilled_parts":pt_col_name})
                    # FULL - break down by both FG and PT 
                    full_serviceability = df_full.groupby(['Finished Good','Item_Number'])[['fulfilled_parts']].mean()
                    full_serviceability.reset_index(level=0, inplace=True)
                    full_serviceability = full_serviceability.rename(columns={"fulfilled_parts":pt_col_name})
                # Otherwise create new information and merge onto existing fg_ and pt_serviceability DataFrames
                else:
                    # FINISHED GOODS level
                    fg_new = fg_fulfillment.groupby('Finished Good')[['fulfilled_parts']].mean()
                    fg_new.reset_index(level=0, inplace=True)
                    fg_serviceability = fg_serviceability.merge(fg_new, left_on='Finished Good', right_on='Finished Good', how='outer')
                    fg_serviceability = fg_serviceability.rename(columns={"fulfilled_parts":fg_col_name})
                    # Clear Memory
                    del fg_new, fg_fulfillment
                    # PARTS level
                    pt_new = df_full.groupby(['Item_Number'])[['fulfilled_parts']].mean()
                    pt_new.reset_index(level=0, inplace=True)
                    pt_serviceability = pt_serviceability.merge(pt_new, left_on='Item_Number', right_on='Item_Number', how='outer')
                    pt_serviceability = pt_serviceability.rename(columns={"fulfilled_parts":pt_col_name})
                    # Clear Memory
                    del pt_new
                    # FULL - break down by both FG and PT
                    full_new = df_full.groupby(['Finished Good','Item_Number'])[['fulfilled_parts']].mean()
                    full_new.reset_index(level=0, inplace=True)
                    full_serviceability = full_serviceability.merge(full_new, on=['Finished Good','Item_Number'], how='outer')
                    full_serviceability = full_serviceability.rename(columns={"fulfilled_parts":pt_col_name})
                    del full_new

                print("--- {}s seconds for simulation {} ---".format((time.time() - start_time), (sim_number+1)))

            # Create fg_, pt_, and full_serviceability filename's from relevant simulation information
            fg_filename = "FGS_"+str(simulations)+"Simulations_"+str(int(noise*100))+"Noise_"+str(sample_of_orders)+"Orders_Batch"+str(batch+1)+".csv"
            pt_filename = "PTS_"+str(simulations)+"Simulations_"+str(int(noise*100))+"Noise_"+str(sample_of_orders)+"Orders_Batch"+str(batch+1)+".csv"
            full_filename = "FULL_"+str(simulations)+"Simulations_"+str(int(noise*100))+"Noise_"+str(sample_of_orders)+"Orders_Batch"+str(batch+1)+".csv"
            # Join fg_ and pt_filename's with PATH_TO_OUTPUT, specified in first cell above
            PATH_FG_OUT = os.path.join(PATH_TO_OUTPUT, fg_filename)
            PATH_PT_OUT = os.path.join(PATH_TO_OUTPUT, pt_filename)
            PATH_FL_OUT = os.path.join(PATH_TO_OUTPUT, full_filename)
            # Output our simulated serviceability data as csv files
            fg_serviceability.to_csv(PATH_FG_OUT)
            pt_serviceability.to_csv(PATH_PT_OUT)
            full_serviceability.to_csv(PATH_FL_OUT)
            # Clear memory and output
            del fg_serviceability, pt_serviceability, full_serviceability
            clear_output(wait=True)

## Run function  

In [3]:
run_simulations(number_of_simulations, noise_ratios, batches)
print('=== Simulations Finished ===')

=== Simulations Finished ===
